In [ ]:

import pandas as pd
pd.set_option('display.max_colwidth', None)
import matplotlib.pyplot as plt
%matplotlib inline
import json
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
train_x=pd.read_csv('../kaggle_data/train_x.csv',index_col=0)
train_y_all=pd.read_csv('../kaggle_data/train_y.csv')
val_x=pd.read_csv('../kaggle_data/val_x.csv',index_col=0)
val_y_all=pd.read_csv('../kaggle_data/val_y.csv')
test_x=pd.read_csv('../kaggle_data/test_x.csv',index_col=0)

In [ ]:
train_x.fillna('',inplace=True)
test_x.fillna('',inplace=True)

In [ ]:
with open('../params.json', 'r') as f:
    params = json.load(f)

max_length = params['max_length']
padding_type = params['padding_type']
vocab_size = params['vocab_size']
embedding_dim = params['embedding_dim']
trunc_type = params['trunc_type']
oov_tok = params['oov_tok']

In [ ]:
train_data.shape

In [ ]:
train_y_all.shape

In [ ]:
train_x.isnull().sum()

In [ ]:
train_y_all.isnull().sum()

In [ ]:
train_x[train_x.isna().any(axis=1)]

In [ ]:
# train_x.dropna(inplace=True)
# train_y_all.drop(141037,inplace=True)

In [ ]:
train_x.shape

In [ ]:
train_y_all.shape

In [ ]:
train_x.head()

In [ ]:
train_y_all['y'].value_counts()

In [ ]:
train_y_all.head()

In [ ]:
val_x.isna().sum()

In [ ]:
val_y_all.isna().sum()

In [ ]:
test_x.isna().sum()

In [ ]:
train_text=train_x['string'].tolist()
val_text=val_x['string'].tolist()
test_text=test_x['string'].tolist()

In [ ]:
train_demo=train_y_all.iloc[:,:8]
val_demo=val_y_all.iloc[:,:8]

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_text)

word_index = tokenizer.word_index


In [ ]:

train_seq = tokenizer.texts_to_sequences(train_text)
val_seq = tokenizer.texts_to_sequences(val_text)
test_seq=tokenizer.texts_to_sequences(test_text)

In [ ]:
word_counts = tokenizer.word_counts
sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
top_n_words = 20000 - 1

top_words_indices = {word: index for index, (word, _) in enumerate(sorted_word_counts[:top_n_words])}
filtered_word_index = {word: index for word, index in tokenizer.word_index.items() if word in top_words_indices}

In [ ]:
train_y=np.array(train_y_all['y'])
val_y=np.array(val_y_all['y'])

In [ ]:
train_x.iloc[0,0]

In [ ]:
total_num_words = [len(one_comment) for one_comment in train_seq]

In [ ]:
plt.hist(total_num_words,bins = np.arange(0,220,10))
plt.show()

In [ ]:
train_padded = pad_sequences(train_seq, maxlen=max_length, padding=padding_type,truncating=trunc_type)
val_padded=pad_sequences(val_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
test_padded=pad_sequences(test_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type) 

In [ ]:
train_padded[0]

In [ ]:
train_y=np.array(train_y)
val_y=np.array(val_y)

In [ ]:
with open('../processed/train_padded.npy', 'wb') as f:
    np.save(f, train_padded)
with open('../processed/val_padded.npy', 'wb') as f:
    np.save(f, val_padded)
with open('../processed/test_padded.npy', 'wb') as f:
    np.save(f, test_padded)
with open('../processed/train_y.npy', 'wb') as f:
    np.save(f, train_y)
with open('../processed/val_y.npy', 'wb') as f:
    np.save(f, val_y)
train_y_all.to_csv('../processed/train_meta.csv',index=False)
val_y_all.to_csv('../processed/val_meta.csv',index=False)
with open('../processed/word_index.json', 'w') as f:
    json.dump(filtered_word_index, f)